In [6]:
import os, sys

project_root = os.path.abspath(os.path.join(os.getcwd(), '..', '..'))
os.environ['PYTHONPATH'] = project_root
if project_root not in sys.path:
    sys.path.insert(0, project_root)

print("PYTHONPATH manually set to:", os.environ['PYTHONPATH'])

PYTHONPATH manually set to: c:\Users\ndhaj\Desktop


## Coded Diffraction Pattern

In [7]:
import numpy as np
from gpie import Graph, GaussianPrior, fft2, AmplitudeMeasurement, pmse
from gpie.core.linalg_utils import random_phase_mask, random_normal_array

H, W = 256, 256
shape = (H, W)
var = 1e-4
rng = np.random.default_rng(seed=123)

class CDPGraph(Graph):
    def __init__(self, num_masks):
        super().__init__()
        x = ~GaussianPrior(var=1.0, shape=shape, label = "object")
        for _ in range(num_masks):
            mask = random_normal_array(shape = shape, rng = rng)
            with self.observe():
                meas = AmplitudeMeasurement(var = var, damping = 0.3) @ (fft2(mask * x))
        self.compile()

g = CDPGraph(num_masks = 3)
g.set_init_rng(np.random.default_rng(seed=999))
g.generate_sample(rng=np.random.default_rng(seed=100), update_observed=True)
true_x = g.get_wave("object").get_sample()

def monitor(graph, t):
    est = graph.get_wave("object").compute_belief().data
    err = pmse(est, true_x)
    if t % 20 == 0:
        print(f"[t={t}] PMSE = {err:.5e}")

g.run(n_iter=200, callback=monitor,verbose = False)

[t=0] PMSE = 1.21984e+00
[t=20] PMSE = 1.00270e+00
[t=40] PMSE = 9.75387e-01
[t=60] PMSE = 6.39233e-01
[t=80] PMSE = 1.82723e-03
[t=100] PMSE = 4.08121e-04
[t=120] PMSE = 4.05713e-04
[t=140] PMSE = 4.05709e-04
[t=160] PMSE = 4.05710e-04
[t=180] PMSE = 4.05710e-04
